# <p style="text-align: center;">MIS 284N: Analytics for Unstructured Data</p>
# <p style="text-align: center;">Assignment 2 (Crowdsourced Recommendation System)</p>
## <p style="text-align: center;">Ally McNulty (agm3734), Chaitra Setty (cs63687), Emilio Cabrera (eac4622), Vishu Agarwal (va7729), Yashpreet Kaur (yk8742)</p>

In [217]:
###################################### loading all the required libraries ################################################
import nltk
import pandas as pd
import nltk
from string import punctuation
from nltk.corpus import stopwords
stop_words = set( stopwords.words('english'))
from nltk.tokenize import word_tokenize
import numpy as np
import string
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
#!python -m spacy download en_core_web_md

## Task A: 

Extract about 5-6k reviews.

In [218]:
################################### loading and cleaning the scraped data ################################################
df = pd.read_csv('beer.csv')

#removing unnecessary columns 
df = df.drop(columns=['web-scraper-order','web-scraper-start-url','beer_link-href','title'])

#renaming the columns 
df = df.rename(columns = {'beer_link' : 'title'})

#cleaning the reviews

###### removing punctuations 
df['cleaned_review'] = df['reviews'].apply(lambda x :str(x).translate(str.maketrans('', '', string.punctuation)))

###### converting the characters to lower case 
df['cleaned_review'] = df['cleaned_review'].apply(lambda x :x.lower())

###### tokenizing the reviews 
df['cleaned_review'] = df['cleaned_review'].apply(word_tokenize).apply(set).apply(list)

###### removing stop words from the reviews token 
def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 
    
df['cleaned_review'] =  df['cleaned_review'].apply(remove_stopwords)
df

,title,reviews,rating,cleaned_review
0,Last Buffalo In The Park,AshBlackstone from Florida\n\n1.34/5 rDev -70...,4.56,"[robust, expect, florida, light, full, 100, le..."
1,Black & Wild,lupercmda from North Carolina\n\n4.77/5 rDev ...,4.53,"[type, sour, finish, 4775, cherry, mid, 45, th..."
2,Supplication,Herbnxplorer from California\n\n4.88/5 rDev +...,4.60,"[across, sour, food, cherry, really, first, ch..."
3,Fort Point Pale Ale - Mosaic Dry Hopped,AZBeerDude72 from Arizona\n\n4.25/5 rDev -5.8...,4.51,"[azbeerdude72, biscuit, nice, jan, medium, lac..."
4,DFPF,BeerSamurai34 from Texas\n\n4.78/5 rDev +5.5%...,4.53,"[beersamurai34, sour, perfectly, time, 45, loo..."
...,...,...,...,...
6222,King Julius,Tony210 from New Jersey\n\n4.48/5 rDev -5.7%\...,4.75,"[16, color, ample, great, flavorful, straw, li..."
6223,Gggreennn!,digboy from New Hampshire\n\n4.63/5 rDev +1.3...,4.57,"[finish, color, dense, light, 45, lacing, look..."
6224,Citra,RaulMondesi from California\n\n4.55/5 rDev -1...,4.60,"[first, jolted, 45, dialed, look, woody, woo, ..."
6225,King Sue - Double Dry-Hopped,DokiDokiLitFam from New Jersey\n\n4.45/5 rDev...,4.56,"[sue, expect, 4, bitter, 45, three, look, ling..."


## Task B: 

Assume that a customer, who will be using this recommender system, has specified 3 attributes in a product. E.g., one website describes multiple attributes of beer:
https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/

A word frequency analysis of beer reviews may be a better way to find important attributes. 
Assume that a customer has specified three attributes of the product as being important to him or her.


In [227]:
##################################### performing word frequency analysis ################################################
#creating bag of words of all the tokens 
tokens_list = df.cleaned_review.sum()

#creating frequency distribution of all the tokens 
freq = nltk.FreqDist(tokens_list)

#identifying top 100 frequently used words 
top_words = freq.most_common(100)

#choosing 3 attributes based on most frequently occuring words 
attr_list = ["dark", "mouthfeel", "balanced"]

In [216]:
top_words

[('rdev', 6227),
 ('taste', 6225),
 ('smell', 6225),
 ('overall', 6225),
 ('look', 6224),
 ('feel', 6224),
 ('45', 4378),
 ('head', 3517),
 ('425', 3061),
 ('475', 3049),
 ('beer', 2779),
 ('4', 1970),
 ('pours', 1780),
 ('nose', 1736),
 ('sweet', 1683),
 ('one', 1679),
 ('dark', 1662),
 ('5', 1650),
 ('carbonation', 1637),
 ('like', 1619),
 ('chocolate', 1608),
 ('2021', 1537),
 ('finish', 1526),
 ('good', 1517),
 ('mouthfeel', 1506),
 ('aroma', 1477),
 ('nice', 1438),
 ('body', 1434),
 ('well', 1432),
 ('2019', 1417),
 ('bottle', 1412),
 ('lacing', 1411),
 ('black', 1394),
 ('2020', 1351),
 ('light', 1335),
 ('notes', 1330),
 ('vanilla', 1307),
 ('medium', 1275),
 ('white', 1248),
 ('smooth', 1239),
 ('flavor', 1213),
 ('bit', 1203),
 ('glass', 1202),
 ('bourbon', 1186),
 ('color', 1175),
 ('fruit', 1161),
 ('great', 1160),
 ('thick', 1152),
 ('really', 1151),
 ('poured', 1137),
 ('2018', 1132),
 ('coffee', 1107),
 ('little', 1082),
 ('orange', 1066),
 ('flavors', 1050),
 ('brown', 1

## Task C: 

Perform a similarity analysis using cosine similarity (without word embeddings) with the 3 attributes specified by the customer and the reviews. From the output file, calculate the average similarity between each product and the preferred attributes. 
For similarity analysis, use cosine similarity with bag of words. The script should accept as input a file with the product attributes, and calculate similarity scores (between 0 and 1) between these attributes and each review. That is, the output file should have 3 columns – product_name (for each product, the product_name will repeat as many times as there are reviews of the product), product_review and similarity_score.

In [228]:
############################### calculating cosine similarity based on bag of words ################################################
attribute = ", ".join(attr_list)

#defining a function to calculate cosine similarity between attributes and reviews 
def similarity(list_of_words):
    string = ", ".join(list_of_words)
    documents = [attribute, string]
    count_vectorizer = CountVectorizer()
    count_matrix = count_vectorizer.fit_transform(documents)
    return cosine_similarity(count_matrix[0:1], count_matrix)[0][1]

#calculating the similarity score
df["similarity"] = df["cleaned_review"].map(similarity)
df_task_c = df[['title', 'reviews','similarity']]
df_task_c  = df_task_c.rename(columns = {'title' : 'product_name', 'reviews' : 'product_review', 'similarity' : 'similarity_score'})
df_task_c

,product_name,product_review,similarity_score
0,Last Buffalo In The Park,AshBlackstone from Florida\n\n1.34/5 rDev -70...,0.000000
1,Black & Wild,lupercmda from North Carolina\n\n4.77/5 rDev ...,0.000000
2,Supplication,Herbnxplorer from California\n\n4.88/5 rDev +...,0.000000
3,Fort Point Pale Ale - Mosaic Dry Hopped,AZBeerDude72 from Arizona\n\n4.25/5 rDev -5.8...,0.000000
4,DFPF,BeerSamurai34 from Texas\n\n4.78/5 rDev +5.5%...,0.000000
...,...,...,...
6222,King Julius,Tony210 from New Jersey\n\n4.48/5 rDev -5.7%\...,0.000000
6223,Gggreennn!,digboy from New Hampshire\n\n4.63/5 rDev +1.3...,0.073922
6224,Citra,RaulMondesi from California\n\n4.55/5 rDev -1...,0.000000
6225,King Sue - Double Dry-Hopped,DokiDokiLitFam from New Jersey\n\n4.45/5 rDev...,0.049875


## Task D: 

For every review, perform a sentiment analysis. 

In [229]:
############################### calculating sentiment score of each review ################################################
analyser = SentimentIntensityAnalyzer()

#defining a function to calculate the sentiment score
def sentiment_analyzer_scores(sentence):
    """Generate seniment score"""
    scores = analyser.polarity_scores(sentence)
    return scores['compound']

df['sentiment'] = df['reviews'].map(sentiment_analyzer_scores)
df

,title,reviews,rating,cleaned_review,similarity,sentiment,spacy_similarity,attribute_presence_1,attribute_presence_2,attribute_presence_3
0,Last Buffalo In The Park,AshBlackstone from Florida\n\n1.34/5 rDev -70...,4.56,"[robust, expect, florida, light, full, 100, le...",0.000000,0.4588,0.509978,0,0,0
1,Black & Wild,lupercmda from North Carolina\n\n4.77/5 rDev ...,4.53,"[type, sour, finish, 4775, cherry, mid, 45, th...",0.000000,0.8795,0.584922,0,0,0
2,Supplication,Herbnxplorer from California\n\n4.88/5 rDev +...,4.60,"[across, sour, food, cherry, really, first, ch...",0.000000,0.9957,0.503785,0,0,0
3,Fort Point Pale Ale - Mosaic Dry Hopped,AZBeerDude72 from Arizona\n\n4.25/5 rDev -5.8...,4.51,"[azbeerdude72, biscuit, nice, jan, medium, lac...",0.000000,0.8725,0.549742,0,0,0
4,DFPF,BeerSamurai34 from Texas\n\n4.78/5 rDev +5.5%...,4.53,"[beersamurai34, sour, perfectly, time, 45, loo...",0.000000,0.8020,0.422907,0,0,0
...,...,...,...,...,...,...,...,...,...,...
6222,King Julius,Tony210 from New Jersey\n\n4.48/5 rDev -5.7%\...,4.75,"[16, color, ample, great, flavorful, straw, li...",0.000000,0.9274,0.652136,1,0,1
6223,Gggreennn!,digboy from New Hampshire\n\n4.63/5 rDev +1.3...,4.57,"[finish, color, dense, light, 45, lacing, look...",0.073922,0.9063,0.583040,0,0,1
6224,Citra,RaulMondesi from California\n\n4.55/5 rDev -1...,4.60,"[first, jolted, 45, dialed, look, woody, woo, ...",0.000000,0.7496,0.453740,0,0,0
6225,King Sue - Double Dry-Hopped,DokiDokiLitFam from New Jersey\n\n4.45/5 rDev...,4.56,"[sue, expect, 4, bitter, 45, three, look, ling...",0.049875,0.9841,0.561733,0,0,0


## Task E: 

Assume an evaluation score for each beer = average similarity score + average sentiment score. 
Now recommend 3 products to the customer. 

In [230]:
############################ providing beer recommendation based on cosine similarity ################################################
combined_reviews_cosine = df.groupby('title')[['similarity','sentiment']].mean()
combined_reviews_cosine['recommend'] = combined_reviews_cosine['similarity']+combined_reviews['sentiment']
recommended_cosine = combined_reviews_cosine.sort_values(by='recommend', ascending=False)[0:3]
recommended_cosine

,similarity,sentiment,recommend
title,,,
Expedition Stout - Bourbon Barrel-Aged,0.083112,0.899836,0.982948
Mother Of All Storms,0.067683,0.903976,0.971659
Genealogy Of Morals - Bourbon Barrel-Aged,0.065885,0.904744,0.970629


## Task F: 

How would your recommendation change if you use word vectors (the spaCy package would be the easiest to use with pretrained word vectors) instead of plain vanilla bag-of-words cosine similarity? One way to analyze the difference would be to consider the % of reviews that mention a preferred attribute. E.g., if you recommend a product, what % of its reviews mention an attribute specified by the customer? Do you see any difference across bag-of-words and word vector approaches? This article may be useful: https://medium.com/swlh/word-embeddings-versus-bag-of-words-the-curious-case-of-recommender-systems-6ac1604d4424?source=friends_link&sk=d746da9f094d1222a35519387afc6338
Note that the article doesn’t claim that bag-of-words will always be better than word embeddings for recommender systems. It lays out conditions under which it is likely to be the case. That is, depending on the attributes you use, you may or may not see the same effect. 

In [231]:
############################ providing beer recommendation based on spacy ################################################

#calculating similarity score based on spacy 
nlp = spacy.load('en_core_web_md')

#defining a function to calculate spacy similarity 
def spacy_similarity(list_of_words):
    doc1 = nlp(list_of_words)
    doc2 = nlp(' '.join(attr_list))
    return doc1.similarity(doc2)

#calculating spacy similarity 
df['spacy_similarity'] = df['reviews'].apply(spacy_similarity)

# providing the beer recommendation 
combined_reviews_spacy = df.groupby('title')[['spacy_similarity','sentiment']].mean()
combined_reviews_spacy['recommend'] = combined_reviews_spacy['spacy_similarity']+combined_reviews['sentiment']
recommended_spacy = combined_reviews_spacy.sort_values(by='recommend', ascending=False)[0:3]
recommended_spacy

,spacy_similarity,sentiment,recommend
title,,,
Mother Of All Storms,0.650948,0.903976,1.554924
Cable Car Kriek,0.609130,0.911192,1.520322
Genealogy Of Morals - Bourbon Barrel-Aged,0.611784,0.904744,1.516528


In [232]:
############################ analysing the differences between the recommendations ################################################

# defining function to check if attribute is present in review 
def review_attr1(review):
    if attr_list[0] in review:
        return 1 
    else:
        return 0;

def review_attr2(review):
    if attr_list[1] in review:
        return 1 
    else:
        return 0;

def review_attr3(review):
    if attr_list[2] in review:
        return 1 
    else:
        return 0;

# creating columns identifying if attributes are present in review 
df['attribute_presence_1'] = df['reviews'].apply(review_attr1)
df['attribute_presence_2'] = df['reviews'].apply(review_attr2)
df['attribute_presence_3'] = df['reviews'].apply(review_attr3)

# aggregating the attribute presence across beers
combined_reviews_cosine = df.groupby('title')[['similarity','sentiment',]].mean()
combined_reviews_cosine['recommend'] = combined_reviews_cosine['similarity']+combined_reviews['sentiment']
df_aggregated = df.groupby('title')[['attribute_presence_1','attribute_presence_2','attribute_presence_3']].sum()
df_aggregated

# % cosine recommended beer reviews containing the attribute words 
recommended_cosine = recommended_cosine.merge(df_aggregated, left_index = True, right_index = True)
# recommended_cosine['attribute_presence_1'] = recommended_cosine['attribute_presence_1']/25
# recommended_cosine['attribute_presence_2'] = recommended_cosine['attribute_presence_2']/25
# recommended_cosine['attribute_presence_3'] = recommended_cosine['attribute_presence_3']/25
recommended_cosine

,attribute_presence_1,attribute_presence_2,attribute_presence_3
title,,,
A Deal With The Devil,14,4,0
A Deal With The Devil - Double Oak-Aged,7,1,1
Aaron,17,2,0
Abner,4,4,7
Abrasive Ale,3,2,4
...,...,...,...
Westly,1,3,5
Wide Awake It's Morning,9,0,4
Zenne Y Frontera,4,4,7


In [234]:
# % spacy recommended beer reviews containing the attribute words 
recommended_spacy = recommended_spacy.merge(df_aggregated, left_index = True, right_index = True)
recommended_spacy

,spacy_similarity,sentiment,recommend,attribute_presence_1,attribute_presence_2,attribute_presence_3
title,,,,,,
Mother Of All Storms,0.650948,0.903976,1.554924,18,4,2
Cable Car Kriek,0.609130,0.911192,1.520322,2,4,3
Genealogy Of Morals - Bourbon Barrel-Aged,0.611784,0.904744,1.516528,13,7,3


We observe that 2 out 3 beer recommendations are similar based on both cosine and spacy similarity. The 3rd beer recommended based on cosine similarity (Expedition Stout - Bourbon Barrel-Aged) has lot more attribute mentions (17,5,2 for each attribute respectively) than the 3rd beer recomendation based on spacy similarity (Cable Car Kriek) has fewer mentions (2,4,3 for each attribute respectively). Spacy must be looking at words similar to the attributes and hence providing the increased similarity. Hence, lets look at the frequently occuring words across reviews for Cable Car Kriek and see what words might be similar to the attributes.

In [283]:
############################## identifying spacy similar words to the attributes ################################################
#creating bag of words of all the tokens across the reviews
tokens_list = df[df['title'] == 'Cable Car Kriek']['cleaned_review'].sum()

#identifying the spacy similar words 
for i in attr_list:
    print ('\033[1m' + 'Spacy similar words to {}'.format(i) + '\033[0m')
    ms = nlp.vocab.vectors.most_similar(np.asarray([nlp.vocab.vectors[nlp.vocab.strings[i]]]), n=30)
    words = [nlp.vocab.strings[w] for w in ms[0][0]]
    distances = ms[2]
    for x in words:
        if x in tokens_list:
            print(x)

Spacy similar words to dark
dark
brown
light
red
white
Spacy similar words to mouthfeel
ale
beer
vanilla
spice
taste
Spacy similar words to balanced
balance
nicely
fairly


We can see that spacy is assessing the similarity based on a lot more words and hence the increased similarity. 

## Task G: 

How would your recommendations differ if you ignored the similarity and feature sentiment scores and simply chose the 3 highest rated products from your entire dataset? Would these products meet the requirements of the user looking for recommendations? Why or why not? Justify your answer with analysis. Use the similarity and sentiment scores as well as overall ratings to answer this question. 
Here is a sample web implementation of a recommender system based on the same principles (runningshoe4you.com), but in this assignment, we will not have the time for this type of full automation.


In [300]:
############################## identifying top 3 beers based on rating ################################################
recommended_rating = df.groupby('title')[['rating','similarity','spacy_similarity','sentiment']].mean().sort_values(by = 'rating', ascending = False)[:3]
recommended_rating

,rating,similarity,spacy_similarity,sentiment
title,,,,
Kentucky Brunch Brand Stout,4.90,0.060531,0.568669,0.686516
Vanilla Bean Assassin,4.85,0.034987,0.565193,0.826438
Marshmallow Handjee,4.81,0.043323,0.537350,0.745952


Since, the recommendation are simply based on the ratings, they might not necessarily be based on the attributes. We can also infer this from the low cosine and spacy similarity scores. Lets look at the most frequently occuring attributes across the reviews of these 3 beers. 

In [298]:
##################################### performing word frequency analysis ################################################
#creating bag of words of all the tokens 
tokens_list = df.merge(recommended_rating, left_on = 'title', right_index = True).cleaned_review.sum()

#creating frequency distribution of all the tokens 
freq = nltk.FreqDist(tokens_list)

#identifying top 100 frequently used words 
top_words = freq.most_common(100)
top_words

[('look', 66),
 ('taste', 66),
 ('overall', 66),
 ('rdev', 66),
 ('feel', 66),
 ('smell', 66),
 ('5', 50),
 ('475', 40),
 ('vanilla', 36),
 ('beer', 34),
 ('bourbon', 29),
 ('2020', 26),
 ('maple', 25),
 ('45', 25),
 ('coffee', 24),
 ('2021', 24),
 ('2019', 24),
 ('thick', 23),
 ('chocolate', 23),
 ('barrel', 23),
 ('black', 22),
 ('like', 21),
 ('dark', 21),
 ('head', 20),
 ('55', 19),
 ('illinois', 17),
 ('4', 16),
 ('best', 16),
 ('bottle', 16),
 ('nose', 15),
 ('jan', 15),
 ('flavors', 14),
 ('glass', 14),
 ('mouthfeel', 14),
 ('2', 13),
 ('stout', 13),
 ('syrup', 13),
 ('’', 13),
 ('2018', 13),
 ('really', 12),
 ('sweet', 12),
 ('good', 12),
 ('flavor', 12),
 ('smooth', 12),
 ('one', 12),
 ('bit', 11),
 ('sweetness', 11),
 ('pour', 11),
 ('vintage', 11),
 ('base', 10),
 ('oil', 9),
 ('much', 9),
 ('great', 9),
 ('dec', 9),
 ('18', 9),
 ('finish', 9),
 ('well', 9),
 ('hype', 9),
 ('worth', 8),
 ('425', 8),
 ('super', 8),
 ('time', 8),
 ('long', 8),
 ('motor', 8),
 ('beers', 8),
 ('

Based on our analysis, we received different results than the top rated beers because the word frequency words are different than the attributes we selected to identify cosine similarity. The frequent words for the top rated beers were maple, vanilla, bourbon, and chocolate versus the attributes we used were mouthful, dark, and balanced. Therefore, we see from this that the top rated beers may have high ratings they may not be the best ratings for the qualities a user would be interested. In conclusion, a cosine similarity and sentiment analysis would be more accurate to provide recommendations rather than arbitrarily selecting the top rated